In [ ]:
from __future__ import print_function
import cv2 as cv
import numpy as np
import random as rng

src = cv.imread('D:/Projects/CV2/Image-Processing/Transformations/balls.jpg')

rng.seed(12345)

def thresh_callback(val):
    threshold = val

    # Detect edges using Canny
    canny_output = cv.Canny(src_gray, threshold, threshold * 2)

    # Find contours
    contours, hierarchy = cv.findContours(canny_output, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    # Draw contours
    drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
    for i in range(len(contours)):
        color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
        cv.drawContours(drawing, contours, i, color, 2, cv.LINE_8, hierarchy, 0)

    # Show in a window
    cv.imshow('Contours', drawing)

# Convert image to gray and blur it
src_gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
src_gray = cv.blur(src_gray, (3,3))

# Create Window
source_window = 'Source'
cv.namedWindow(source_window)
cv.imshow(source_window, src)
max_thresh = 255
thresh = 100 # initial threshold
cv.createTrackbar('Canny Thresh:', source_window, thresh, max_thresh, thresh_callback)
thresh_callback(thresh)

cv.waitKey()

In [16]:
#мій код
from __future__ import print_function
import cv2 as cv
import numpy as np
import random as rng

img = cv.imread('tank.jpg', cv.IMREAD_GRAYSCALE)

img = cv.resize(img, None, fx=2, fy=2, interpolation=cv.INTER_AREA)

threshold = 140

canny = cv.Canny(img, threshold, threshold * 2)

contours, hierarchy = cv.findContours(canny, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

drawing = np.zeros((canny.shape[0], canny.shape[1], 3), dtype=np.uint8)
for i in range(len(contours)):
    color = (0,255,0)
    cv.drawContours(drawing, contours, i, color, 2, cv.LINE_8, hierarchy, 0)

cv.imshow('contours', drawing)
cv.waitKey(0)
cv.destroyAllWindows()

**Convex Hull**

In [6]:
from __future__ import print_function
import cv2 as cv
import numpy as np
import random as rng

rng.seed(12345)

def thresh_callback(val):
    threshold = val

    # Detect edges using Canny
    canny_output = cv.Canny(src_gray, threshold, threshold * 2)

    # Find contours
    contours, _ = cv.findContours(canny_output, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    # Find the convex hull object for each contour
    hull_list = []
    for i in range(len(contours)):
        hull = cv.convexHull(contours[i])
        hull_list.append(hull)

    # Draw contours + hull results
    drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
    for i in range(len(contours)):
        color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
        cv.drawContours(drawing, contours, i, color)
        cv.drawContours(drawing, hull_list, i, color)

    # Show in a window
    cv.imshow('Contours', drawing)

src = cv.imread('hands.jpg')

# Convert image to gray and blur it
src_gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
src_gray = cv.blur(src_gray, (3,3))

# Create Window
source_window = 'Source'
cv.namedWindow(source_window)
cv.imshow(source_window, src)
max_thresh = 255
thresh = 100 # initial threshold
cv.createTrackbar('Canny thresh:', source_window, thresh, max_thresh, thresh_callback)
thresh_callback(thresh)

cv.waitKey()

-1

In [14]:
#мій код
from __future__ import print_function
import cv2 as cv
import numpy as np
import random as rng

img = cv.imread('hands.jpg')

#img = cv.resize(img, None, fx=2, fy=2, interpolation=cv.INTER_AREA)

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

blur = cv.blur(gray, (3,3))

threshold = 80

canny = cv.Canny(blur, threshold, threshold * 2)

contours, _ = cv.findContours(canny, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

hull_list = []
for i in range(len(contours)):
    hull = cv.convexHull(contours[i])
    hull_list.append(hull)

drawing = np.zeros((canny.shape[0], canny.shape[1], 3), dtype=np.uint8)
for i in range(len(contours)):
    cv.drawContours(drawing, contours, i, (0,255,0), 2)
    cv.drawContours(drawing, hull_list, i, (0,0,255), 1)

cv.imshow('hull', drawing)
cv.imshow('original', img)
cv.waitKey(0)
cv.destroyAllWindows()

**Creating Bounding boxes and circles for contours**

In [ ]:
from __future__ import print_function
import cv2 as cv
import numpy as np
import argparse
import random as rng

rng.seed(12345)

def thresh_callback(val):
    threshold = val

    canny_output = cv.Canny(src_gray, threshold, threshold * 2)

    contours, _ = cv.findContours(canny_output, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    contours_poly = [None]*len(contours)
    boundRect = [None]*len(contours)
    centers = [None]*len(contours)
    radius = [None]*len(contours)
    for i, c in enumerate(contours):
        contours_poly[i] = cv.approxPolyDP(c, 3, True)
        boundRect[i] = cv.boundingRect(contours_poly[i])
        centers[i], radius[i] = cv.minEnclosingCircle(contours_poly[i])

    drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)

    for i in range(len(contours)):
        color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
        cv.drawContours(drawing, contours_poly, i, color)
        cv.rectangle(drawing, (int(boundRect[i][0]), int(boundRect[i][1])), \
          (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)
        cv.circle(drawing, (int(centers[i][0]), int(centers[i][1])), int(radius[i]), color, 2)

    cv.imshow('Contours', drawing)

src = cv.imread('D:/Projects/CV2/Image-Processing/Transformations/balls.jpg')

# Convert image to gray and blur it
src_gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
src_gray = cv.blur(src_gray, (3,3))

source_window = 'Source'
cv.namedWindow(source_window)
cv.imshow(source_window, src)

max_thresh = 255
thresh = 60 # initial threshold
cv.createTrackbar('Canny thresh:', source_window, thresh, max_thresh, thresh_callback)
thresh_callback(thresh)

cv.waitKey()

-1

In [3]:
#мій код
from __future__ import print_function
import cv2 as cv
import numpy as np
import random as rng

img = cv.imread('D:/Projects/CV2/Image-Processing/Transformations/balls.jpg')

img = cv.resize(img, None, fx=0.7, fy=0.7, interpolation=cv.INTER_AREA)

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

threshold = 100

canny = cv.Canny(gray, threshold, threshold * 2)

contours, hierarchy = cv.findContours(canny, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

contours_poly = [None]*len(contours)
boundRect = [None]*len(contours)
centers = [None]*len(contours)
radius = [None]*len(contours)
for i, c in enumerate(contours):
    contours_poly[i] = cv.approxPolyDP(c, 3, True)
    boundRect[i] = cv.boundingRect(contours_poly[i])
    centers[i], radius[i] = cv.minEnclosingCircle(contours_poly[i])

drawing = np.zeros((canny.shape[0], canny.shape[1], 3), dtype=np.uint8)

for i in range(len(contours)):
        #cv.drawContours(drawing, contours_poly, i, (255,0,0))
        cv.rectangle(img, (int(boundRect[i][0]), int(boundRect[i][1])), \
        (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), (0,0,255), 2)
        cv.circle(img, (int(centers[i][0]), int(centers[i][1])), int(radius[i]), (0,255,0), 2)

cv.imshow('contours', img)
cv.waitKey(0)
cv.destroyAllWindows()

**Creating Bounding rotated boxes and ellipses for contours**

In [5]:
from __future__ import print_function
import cv2 as cv
import numpy as np
import random as rng

rng.seed(12345)

def thresh_callback(val):
    threshold = val

    canny_output = cv.Canny(src_gray, threshold, threshold * 2)

    contours, _ = cv.findContours(canny_output, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    # Find the rotated rectangles and ellipses for each contour
    minRect = [None]*len(contours)
    minEllipse = [None]*len(contours)
    for i, c in enumerate(contours):
        minRect[i] = cv.minAreaRect(c)
        if c.shape[0] > 100:
            minEllipse[i] = cv.fitEllipse(c)

    # Draw contours + rotated rects + ellipses

    drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)

    for i, c in enumerate(contours):
        color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
        # contour
        cv.drawContours(drawing, contours, i, color)
        # ellipse
        if c.shape[0] > 100:
            cv.ellipse(drawing, minEllipse[i], color, 2)
        # rotated rectangle
        box = cv.boxPoints(minRect[i])
        box = np.intp(box) #np.intp: Integer used for indexing (same as C ssize_t; normally either int32 or int64)
        cv.drawContours(drawing, [box], 0, color)

    cv.imshow('Contours', drawing)

src = cv.imread('air_balloon.jpg')

# Convert image to gray and blur it
src_gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
#src_gray = cv.blur(src_gray, (3,3))

source_window = 'Source'
cv.namedWindow(source_window)
cv.imshow(source_window, src)

max_thresh = 255
thresh = 100 # initial threshold
cv.createTrackbar('Canny Thresh:', source_window, thresh, max_thresh, thresh_callback)
thresh_callback(thresh)

cv.waitKey()

-1

**Image Moments**

In [1]:
from __future__ import print_function
from __future__ import division
import cv2 as cv
import numpy as np
import argparse
import random as rng

rng.seed(12345)

def thresh_callback(val):
    threshold = val

    canny_output = cv.Canny(src_gray, threshold, threshold * 2)

    contours, _ = cv.findContours(canny_output, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    # Get the moments
    mu = [None]*len(contours)
    for i in range(len(contours)):
        mu[i] = cv.moments(contours[i])

    # Get the mass centers
    mc = [None]*len(contours)
    for i in range(len(contours)):
        # add 1e-5 to avoid division by zero
        mc[i] = (mu[i]['m10'] / (mu[i]['m00'] + 1e-5), mu[i]['m01'] / (mu[i]['m00'] + 1e-5))

    # Draw contours

    drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)

    for i in range(len(contours)):
        color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
        cv.drawContours(drawing, contours, i, color, 2)
        cv.circle(drawing, (int(mc[i][0]), int(mc[i][1])), 4, color, -1)
        cv.imshow('result', drawing)

    

    # Calculate the area with the moments 00 and compare with the result of the OpenCV function
    for i in range(len(contours)):
        print(' * Contour[%d] - Area (M_00) = %.2f - Area OpenCV: %.2f - Length: %.2f' % (i, mu[i]['m00'], cv.contourArea(contours[i]), cv.arcLength(contours[i], True)))

#src = cv.imread('monsters.jpg')
src = cv.imread('monsters.jpg')

# Convert image to gray and blur it
src_gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
src_gray = cv.blur(src_gray, (3,3))

source_window = 'Source'
cv.namedWindow(source_window)
cv.imshow(source_window, src)


max_thresh = 255
thresh = 100 # initial threshold
cv.createTrackbar('Canny Thresh:', source_window, thresh, max_thresh, thresh_callback)
thresh_callback(thresh)

cv.waitKey()
cv.destroyAllWindows()

 * Contour[0] - Area (M_00) = 42.50 - Area OpenCV: 42.50 - Length: 349.32
 * Contour[1] - Area (M_00) = 99.00 - Area OpenCV: 99.00 - Length: 76.28
 * Contour[2] - Area (M_00) = 92.00 - Area OpenCV: 92.00 - Length: 73.11
 * Contour[3] - Area (M_00) = 58.00 - Area OpenCV: 58.00 - Length: 29.31
 * Contour[4] - Area (M_00) = 50.00 - Area OpenCV: 50.00 - Length: 26.97
 * Contour[5] - Area (M_00) = 72.00 - Area OpenCV: 72.00 - Length: 31.31
 * Contour[6] - Area (M_00) = 62.00 - Area OpenCV: 62.00 - Length: 28.97
 * Contour[7] - Area (M_00) = 61.00 - Area OpenCV: 61.00 - Length: 29.31
 * Contour[8] - Area (M_00) = 53.00 - Area OpenCV: 53.00 - Length: 26.97
 * Contour[9] - Area (M_00) = 64.00 - Area OpenCV: 64.00 - Length: 29.31
 * Contour[10] - Area (M_00) = 54.00 - Area OpenCV: 54.00 - Length: 26.97
 * Contour[11] - Area (M_00) = 105.00 - Area OpenCV: 105.00 - Length: 38.14
 * Contour[12] - Area (M_00) = 95.00 - Area OpenCV: 95.00 - Length: 35.80
 * Contour[13] - Area (M_00) = 3853.00 - Area

In [1]:
#мій код
from __future__ import print_function
import cv2 as cv
import numpy as np

img = cv.imread('tank.jpg', cv.IMREAD_GRAYSCALE)

threshold = 100

canny = cv.Canny(img, threshold, threshold * 2)

contours, hierarchy = cv.findContours(canny, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

moments = [None]*len(contours)
for i in range(len(contours)):
    moments[i] = cv.moments(contours[i])

mass_center = [None]*len(contours)
for i in range(len(contours)):
     mass_center[i] = (moments[i]['m10'] / (moments[i]['m00'] + 1e-5), moments[i]['m01'] / (moments[i]['m00'] + 1e-5))

drawing = np.zeros((canny.shape[0], canny.shape[1], 3), dtype=np.uint8)
for i in range(len(contours)):
    color = (0,255,0)
    cv.drawContours(drawing, contours, i, color, 2, cv.LINE_8, hierarchy, 0)

for i in range(len(contours)):
        print(' * Contour[%d] - Area (M_00) = %.2f - Area OpenCV: %.2f - Length: %.2f' % (i, moments[i]['m00'], cv.contourArea(contours[i]), cv.arcLength(contours[i], True)))


cv.imshow('contours', drawing)
cv.waitKey(0)
cv.destroyAllWindows()



 * Contour[0] - Area (M_00) = 0.00 - Area OpenCV: 0.00 - Length: 0.00
 * Contour[1] - Area (M_00) = 0.00 - Area OpenCV: 0.00 - Length: 0.00
 * Contour[2] - Area (M_00) = 0.00 - Area OpenCV: 0.00 - Length: 2.83
 * Contour[3] - Area (M_00) = 0.00 - Area OpenCV: 0.00 - Length: 0.00
 * Contour[4] - Area (M_00) = 1.00 - Area OpenCV: 1.00 - Length: 4.83
 * Contour[5] - Area (M_00) = 0.50 - Area OpenCV: 0.50 - Length: 11.07
 * Contour[6] - Area (M_00) = 0.00 - Area OpenCV: 0.00 - Length: 5.66
 * Contour[7] - Area (M_00) = 0.00 - Area OpenCV: 0.00 - Length: 4.83
 * Contour[8] - Area (M_00) = 0.00 - Area OpenCV: 0.00 - Length: 2.00
 * Contour[9] - Area (M_00) = 0.00 - Area OpenCV: 0.00 - Length: 6.00
 * Contour[10] - Area (M_00) = 0.50 - Area OpenCV: 0.50 - Length: 6.24
 * Contour[11] - Area (M_00) = 0.50 - Area OpenCV: 0.50 - Length: 3.41
 * Contour[12] - Area (M_00) = 0.00 - Area OpenCV: 0.00 - Length: 0.00
 * Contour[13] - Area (M_00) = 4.00 - Area OpenCV: 4.00 - Length: 8.00
 * Contour[14] 

**Point Polygon Test**

In [20]:
from __future__ import print_function
from __future__ import division
import cv2 as cv
import numpy as np

# Create an image
r = 100
src = np.zeros((4*r, 4*r), dtype=np.uint8)

# Create a sequence of points to make a contour
vert = [None]*6
vert[0] = (3*r//2, int(1.34*r))
vert[1] = (1*r, 2*r)
vert[2] = (3*r//2, int(2.866*r))
vert[3] = (5*r//2, int(2.866*r))
vert[4] = (3*r, 2*r)
vert[5] = (5*r//2, int(1.34*r))

# Draw it in src
for i in range(6):
    cv.line(src, vert[i],  vert[(i+1)%6], ( 255 ), 3)

# Get the contours
contours, _ = cv.findContours(src, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

# Calculate the distances to the contour
raw_dist = np.empty(src.shape, dtype=np.float32)
for i in range(src.shape[0]):
    for j in range(src.shape[1]):
        raw_dist[i,j] = cv.pointPolygonTest(contours[0], (j,i), True)

minVal, maxVal, _, maxDistPt = cv.minMaxLoc(raw_dist)
minVal = abs(minVal)
maxVal = abs(maxVal)

# Depicting the  distances graphically
drawing = np.zeros((src.shape[0], src.shape[1], 3), dtype=np.uint8)
for i in range(src.shape[0]):
    for j in range(src.shape[1]):
        if raw_dist[i,j] < 0:
            drawing[i,j,0] = 255 - abs(raw_dist[i,j]) * 255 / minVal
        elif raw_dist[i,j] > 0:
            drawing[i,j,2] = 255 - raw_dist[i,j] * 255 / maxVal
        else:
            drawing[i,j,0] = 255
            drawing[i,j,1] = 255
            drawing[i,j,2] = 255

cv.circle(drawing,maxDistPt, int(maxVal),(255,255,255), 1, cv.LINE_8, 0)
cv.imshow('Source', src)
cv.imshow('Distance and inscribed circle', drawing)
cv.waitKey()

-1

In [28]:
from __future__ import print_function
from __future__ import division
import cv2 as cv
import numpy as np

# Create an image
r = 100
src = np.zeros((4*r, 4*r), dtype=np.uint8)

# Create a sequence of points to make a contour
vert = []
center = (2*r, 2*r) 
radius = int(1.2 * r)  
for i in range(8):
    angle = 2 * np.pi * i / 8
    x = int(center[0] + radius * np.cos(angle))
    y = int(center[1] + radius * np.sin(angle))
    vert.append((x, y))

# Draw it in src
for i in range(8):
    cv.line(src, vert[i],  vert[(i+1)%8], ( 255 ), 3)

contours, _ = cv.findContours(src, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

raw_dist = np.empty(src.shape, dtype=np.float32)
for i in range(src.shape[0]):
    for j in range(src.shape[1]):
        raw_dist[i,j] = cv.pointPolygonTest(contours[0], (j,i), True)

minVal, maxVal, _, maxDistPt = cv.minMaxLoc(raw_dist)
minVal = abs(minVal)
maxVal = abs(maxVal)

drawing = np.zeros((src.shape[0], src.shape[1], 3), dtype=np.uint8)
for i in range(src.shape[0]):
    for j in range(src.shape[1]):
        if raw_dist[i,j] > 0:
            drawing[i,j,2] = 255 - raw_dist[i,j]
        if raw_dist[i,j] < 0:
            drawing[i,j,0] = 255 - abs(raw_dist[i,j])

cv.imshow('Source', src)
cv.imshow('Distance and inscribed circle', drawing)
cv.waitKey()

-1